# 🎵 YouTube Video Transcription Pipeline
## Pixelated Empathy Training Dataset

This notebook transcribes YouTube videos and creates beautiful markdown files for the training dataset.

**Features:**
- High-quality Whisper transcription
- Beautiful markdown formatting
- Batch processing with progress tracking
- Automatic file organization

## 🔧 Setup & Installation

In [ ]:
# Install dependencies
!pip install openai-whisper tqdm

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import json
import shutil
import whisper
import re
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
from zipfile import ZipFile
import urllib.request

## 📁 Upload Audio Files

**Option 1:** Upload a ZIP file containing all MP3s  
**Option 2:** Upload individual MP3 files  
**Option 3:** Use files from Google Drive

In [ ]:
# Create working directories
os.makedirs('/content/audio', exist_ok=True)
os.makedirs('/content/transcripts', exist_ok=True)
os.makedirs('/content/markdown', exist_ok=True)

# Option 1: Upload ZIP file (uncomment to use)
# from google.colab import files
# uploaded = files.upload()
# for filename in uploaded.keys():
#     if filename.endswith('.zip'):
#         with ZipFile(filename, 'r') as zip_ref:
#             zip_ref.extractall('/content/audio')
#         print(f"Extracted {filename}")

# Option 2: Upload individual files (uncomment to use)
# uploaded = files.upload()
# for filename in uploaded.keys():
#     if filename.endswith('.mp3'):
#         shutil.move(filename, f'/content/audio/{filename}')

# Option 3: Copy from Google Drive (modify path as needed)
# drive_path = '/content/drive/MyDrive/youtube_audio'  # Change this path
# if os.path.exists(drive_path):
#     for file in os.listdir(drive_path):
#         if file.endswith('.mp3'):
#             shutil.copy2(f'{drive_path}/{file}', f'/content/audio/{file}')

# List uploaded files
audio_files = [f for f in os.listdir('/content/audio') if f.endswith('.mp3')]
print(f"Found {len(audio_files)} MP3 files")
for f in audio_files[:5]:  # Show first 5
    print(f"  - {f}")
if len(audio_files) > 5:
    print(f"  ... and {len(audio_files) - 5} more")

## 🎤 Transcription Pipeline

In [ ]:
# Load Whisper model
MODEL_SIZE = "base"  # Options: tiny, base, small, medium, large
print(f"Loading Whisper model: {MODEL_SIZE}")
model = whisper.load_model(MODEL_SIZE)
print("Model loaded successfully!")

In [ ]:
def sanitize_filename(filename):
    """Clean filename for safe usage"""
    name = re.sub(r'[<>:"/\\|?*]', '_', filename)
    name = re.sub(r'[^\w\s\-_\.]', '_', name)
    return re.sub(r'\s+', '_', name)[:200]

def extract_metadata(filename):
    """Extract channel and title from filename"""
    name = filename.replace('.mp3', '')
    
    # Try to parse channel and title
    if ' - ' in name:
        parts = name.split(' - ', 1)
        channel = parts[0].strip()
        title = parts[1].strip()
    else:
        channel = "Unknown Channel"
        title = name
    
    return {
        'channel': channel,
        'title': title,
        'filename': filename,
        'processed_date': datetime.now().isoformat()
    }

def create_markdown(transcription, metadata):
    """Create beautiful markdown transcript"""
    
    markdown = f"""# {metadata['title']}

## 📺 Channel Information
**Channel:** {metadata['channel']}  
**File:** `{metadata['filename']}`  
**Processed:** {metadata['processed_date'][:10]}  

## 📊 Transcript Details
- **Language:** {transcription['language'].upper()}
- **Duration:** {sum(seg.get('end', 0) - seg.get('start', 0) for seg in transcription.get('segments', [])):.1f} seconds
- **Word Count:** {len(transcription['text'].split()):,} words

---

## 📝 Full Transcript

{transcription['text']}

---

## 🎯 Detailed Segments

"""
    
    # Add segments
    for i, segment in enumerate(transcription.get('segments', []), 1):
        start = segment.get('start', 0)
        end = segment.get('end', 0)
        text = segment.get('text', '').strip()
        
        start_min, start_sec = divmod(int(start), 60)
        end_min, end_sec = divmod(int(end), 60)
        
        markdown += f"""### Segment {i}
**Time:** {start_min:02d}:{start_sec:02d} - {end_min:02d}:{end_sec:02d}

{text}

"""
    
    markdown += f"""---

## 🔧 Technical Information

- **Model:** Whisper {MODEL_SIZE}
- **Pipeline:** Pixelated Empathy Training Dataset
- **Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

---

*Transcript generated for Pixelated Empathy AI training dataset.*
"""
    
    return markdown

print("Functions defined successfully!")

In [ ]:
# Process all audio files
audio_files = [f for f in os.listdir('/content/audio') if f.endswith('.mp3')]
results = {'successful': 0, 'failed': 0, 'files': []}

print(f"Processing {len(audio_files)} files...\n")

for i, filename in enumerate(tqdm(audio_files, desc="Transcribing")):
    try:
        # Transcribe
        audio_path = f'/content/audio/{filename}'
        result = model.transcribe(audio_path)
        
        # Extract metadata
        metadata = extract_metadata(filename)
        
        # Create clean filename
        clean_name = sanitize_filename(filename.replace('.mp3', ''))
        
        # Save JSON
        json_data = {
            'metadata': metadata,
            'transcription': result,
            'processed_at': datetime.now().isoformat()
        }
        
        json_path = f'/content/transcripts/{clean_name}.json'
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        
        # Save Markdown
        markdown_content = create_markdown(result, metadata)
        markdown_path = f'/content/markdown/{clean_name}.md'
        with open(markdown_path, 'w', encoding='utf-8') as f:
            f.write(markdown_content)
        
        results['successful'] += 1
        results['files'].append({
            'filename': filename,
            'status': 'success',
            'json_path': json_path,
            'markdown_path': markdown_path,
            'language': result.get('language', 'unknown'),
            'duration': sum(seg.get('end', 0) - seg.get('start', 0) for seg in result.get('segments', [])),
            'word_count': len(result['text'].split())
        })
        
        print(f"✅ {filename} - {result.get('language', 'unknown')} - {len(result['text'].split())} words")
        
    except Exception as e:
        results['failed'] += 1
        results['files'].append({
            'filename': filename,
            'status': 'failed',
            'error': str(e)
        })
        print(f"❌ {filename} - Error: {e}")

print(f"\n🎉 Processing complete!")
print(f"✅ Successful: {results['successful']}")
print(f"❌ Failed: {results['failed']}")
print(f"📈 Success Rate: {results['successful']/(results['successful']+results['failed'])*100:.1f}%")

## 📥 Download Results

In [ ]:
# Create summary report
summary = {
    'processing_summary': {
        'total_files': len(audio_files),
        'successful': results['successful'],
        'failed': results['failed'],
        'success_rate': f"{results['successful']/(results['successful']+results['failed'])*100:.1f}%",
        'processed_date': datetime.now().isoformat(),
        'model_used': MODEL_SIZE
    },
    'file_details': results['files'],
    'statistics': {
        'total_words': sum(f.get('word_count', 0) for f in results['files'] if f['status'] == 'success'),
        'total_duration': sum(f.get('duration', 0) for f in results['files'] if f['status'] == 'success'),
        'languages': list(set(f.get('language', 'unknown') for f in results['files'] if f['status'] == 'success'))
    }
}

# Save summary
with open('/content/transcription_summary.json', 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print("📊 Summary Report:")
print(f"Total Words: {summary['statistics']['total_words']:,}")
print(f"Total Duration: {summary['statistics']['total_duration']:.1f} seconds")
print(f"Languages: {', '.join(summary['statistics']['languages'])}")

In [ ]:
# Create ZIP file for download
import zipfile

zip_filename = f'pixelated_transcripts_{datetime.now().strftime("%Y%m%d_%H%M%S")}.zip'

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add all markdown files
    for file in os.listdir('/content/markdown'):
        if file.endswith('.md'):
            zipf.write(f'/content/markdown/{file}', f'markdown/{file}')
    
    # Add all JSON files
    for file in os.listdir('/content/transcripts'):
        if file.endswith('.json'):
            zipf.write(f'/content/transcripts/{file}', f'json/{file}')
    
    # Add summary
    zipf.write('/content/transcription_summary.json', 'transcription_summary.json')

print(f"📦 Created: {zip_filename}")
print(f"📁 Contains: {len(os.listdir('/content/markdown'))} markdown files")
print(f"📄 Contains: {len(os.listdir('/content/transcripts'))} JSON files")

In [ ]:
# Download the ZIP file
from google.colab import files

print("Downloading transcription results...")
files.download(zip_filename)

# Also save to Google Drive (optional)
drive_path = '/content/drive/MyDrive/pixelated_transcripts'
os.makedirs(drive_path, exist_ok=True)
shutil.copy2(zip_filename, f'{drive_path}/{zip_filename}')
print(f"Also saved to Google Drive: {drive_path}/{zip_filename}")

## 👀 Preview Results

In [ ]:
# Show a sample markdown file
markdown_files = [f for f in os.listdir('/content/markdown') if f.endswith('.md')]

if markdown_files:
    sample_file = markdown_files[0]
    print(f"📄 Preview of: {sample_file}")
    print("=" * 60)
    
    with open(f'/content/markdown/{sample_file}', 'r', encoding='utf-8') as f:
        content = f.read()
        # Show first 1000 characters
        print(content[:1000])
        if len(content) > 1000:
            print("\n... (truncated)")
    
    print("\n" + "=" * 60)
    print(f"Full file size: {len(content):,} characters")
else:
    print("No markdown files found.")